### Import Testing

In [1]:
# Importazione delle librerie necessarie
import requests
import yaml as yl
# Verifica delle versioni delle librerie installate
import sys
import pprint
from datetime import datetime
#import landsatxplore for the future
print(f"Requests version: {requests.__version__}")
print(sys.executable)


Requests version: 2.28.1
c:\Users\fgalassi\Progetti\EO Pipeline\processing-root-folder\.venv\Scripts\python.exe


### Test Config 

In [2]:
import os
print("Current working directory:", os.getcwd())
os.chdir('../')
print("Current working directory:", os.getcwd())


Current working directory: c:\Users\fgalassi\Progetti\EO Pipeline\processing-root-folder\notebooks
Current working directory: c:\Users\fgalassi\Progetti\EO Pipeline\processing-root-folder


In [3]:
import os

# Build the path to your configuration file
config_path = os.path.join("config", "config.yaml")

# Check if the configuration file exists
if not os.path.exists(config_path):
    print(f"Configuration file not found at: {config_path}")
else:
    # Open and load the configuration file safely
    with open(config_path, 'r') as file:
        config = yl.safe_load(file)
    
    print("Configuration loaded successfully:")
    print(config)
    
    # Optional: Verify some required keys are in the configuration
    required_keys = ['data', 'satellite', 'download', 'processing', 'logging']
    missing_keys = [key for key in required_keys if key not in config]
    if missing_keys:
        print("Missing expected configuration keys:", missing_keys)
    else:
        print("All expected keys are present in the configuration.")


Configuration loaded successfully:
{'data': {'dates': {'start': '2022-01-01', 'end': '2022-01-31'}, 'polygon': {'type': 'Polygon', 'coordinates': [[[12.3, 41.8], [12.3, 42.0], [12.5, 42.0], [12.5, 41.8], [12.3, 41.8]]]}}, 'download': {'provider': 'usgs', 'copernicus': {'url': 'https://scihub.copernicus.eu/dhus', 'username': 'your_username', 'password': 'your_password', 'cloud_cover_threshold': 30}, 'usgs': {'url': 'https://earthexplorer.usgs.gov/inventory/json/v/1.4.1/', 'api_key': 'your_api_key', 'dataset': 'LANDSAT_8_C1'}}, 'satellite': {'copernicus': {'platform': 'Sentinel-2', 'product_level': 'L2A'}, 'usgs': {'platform': 'Landsat-8', 'product_level': 'L1'}}, 'processing': {'output_folder': 'data/processed', 'intermediate_folder': 'data/intermediate', 'algorithm': 'ndvi'}, 'logging': {'log_file': 'results/logs/pipeline.log', 'log_level': 'INFO'}}
All expected keys are present in the configuration.


## test dati download

In [4]:
# Percorso del file di configurazione
config_path = os.path.join("config", "config.yaml")

# Verifica dell'esistenza del file
if not os.path.exists(config_path):
    print(f"File di configurazione non trovato: {config_path}")
else:
    # Carica la configurazione dal file YAML
    with open(config_path, 'r') as file:
        config = yl.safe_load(file)

    # Maschera le informazioni sensibili per la stampa
    if "download" in config:
        # Maschera la password per Copernicus, se presente
        if "copernicus" in config["download"] and "password" in config["download"]["copernicus"]:
            config["download"]["copernicus"]["password"] = "********"
        # Maschera la API key per USGS, se presente
        if "usgs" in config["download"] and "api_key" in config["download"]["usgs"]:
            config["download"]["usgs"]["api_key"] = "********"

    # Stampa delle informazioni caricate
    print("=== Configurazione del download ===")
    print("\nIntervallo di date:")
    print("  Inizio:", config["data"]["dates"]["start"])
    print("  Fine   :", config["data"]["dates"]["end"])

    print("\nCoordinate del poligono (Area di Interesse):")
    print("  Tipo:", config["data"]["polygon"]["type"])
    print("  Coordinate:")
    pprint.pprint(config["data"]["polygon"]["coordinates"])

    print("\nParametri per il download:")
    provider = config["download"].get("provider", "").lower()
    print("  Provider:", provider)
    if provider == "copernicus":
        copernicus_conf = config["download"].get("copernicus", {})
        print("  URL:", copernicus_conf.get("url", ""))
        print("  Username:", copernicus_conf.get("username", ""))
        print("  Password:", copernicus_conf.get("password", ""))
        print("  Soglia copertura nuvolosa:", copernicus_conf.get("cloud_cover_threshold", ""))
    elif provider == "usgs":
        usgs_conf = config["download"].get("usgs", {})
        print("  URL:", usgs_conf.get("url", ""))
        print("  API Key:", usgs_conf.get("api_key", ""))
        print("  Dataset:", usgs_conf.get("dataset", ""))
    else:
        print("  Provider non riconosciuto.")

    print("\nParametri del satellite:")
    if provider == "copernicus":
        sat_params = config["satellite"].get("copernicus", {})
    elif provider == "usgs":
        sat_params = config["satellite"].get("usgs", {})
    else:
        sat_params = {}
    print("  Piattaforma:", sat_params.get("platform", ""))
    print("  Livello di prodotto:", sat_params.get("product_level", ""))

    print("\nImpostazioni di processing:")
    print("  Cartella di output:", config["processing"].get("output_folder", ""))
    print("  Cartella intermedia:", config["processing"].get("intermediate_folder", ""))
    print("  Algoritmo:", config["processing"].get("algorithm", ""))

    print("\nImpostazioni di logging:")
    print("  File di log:", config["logging"].get("log_file", ""))
    print("  Livello di log:", config["logging"].get("log_level", ""))



=== Configurazione del download ===

Intervallo di date:
  Inizio: 2022-01-01
  Fine   : 2022-01-31

Coordinate del poligono (Area di Interesse):
  Tipo: Polygon
  Coordinate:
[[[12.3, 41.8], [12.3, 42.0], [12.5, 42.0], [12.5, 41.8], [12.3, 41.8]]]

Parametri per il download:
  Provider: usgs
  URL: https://earthexplorer.usgs.gov/inventory/json/v/1.4.1/
  API Key: ********
  Dataset: LANDSAT_8_C1

Parametri del satellite:
  Piattaforma: Landsat-8
  Livello di prodotto: L1

Impostazioni di processing:
  Cartella di output: data/processed
  Cartella intermedia: data/intermediate
  Algoritmo: ndvi

Impostazioni di logging:
  File di log: results/logs/pipeline.log
  Livello di log: INFO


In [5]:
#!/usr/bin/env python
"""
Script per il download dei dati satellitari.
Il file di configurazione (config/config.yaml) deve definire:
- Intervallo di date e area d'interesse (AOI) nel formato GeoJSON.
- Provider di download ("copernicus" oppure "usgs") e i relativi parametri.
- Parametri satellitari specifici per ciascun provider:
    * Nella sezione satellite.copernicus per Sentinel-2,
    * Nella sezione satellite.usgs per Landsat-8 (o altri).
- Se il campo download.dest_folder non è definito, la cartella di download viene impostata in base al product_level.
"""


# Caricamento del file di configurazione
config_path = os.path.join("config", "config.yaml")
if not os.path.exists(config_path):
    raise FileNotFoundError(f"File di configurazione non trovato: {config_path}")

with open(config_path, 'r') as file:
    config = yl.safe_load(file)

# ----- Estrazione dei parametri ----- #
data_conf = config.get("data", {})
download_conf = config.get("download", {})
satellite_conf = config.get("satellite", {})

# Estrazione dell'intervallo di date (formato "YYYY-MM-DD") e conversione in formato "YYYYMMDD"
start_date = data_conf.get("dates", {}).get("start")
end_date = data_conf.get("dates", {}).get("end")
if not start_date or not end_date:
    raise ValueError("Definire le date (start ed end) nella sezione 'data: dates'.")
start_date_api = start_date.replace("-", "")
end_date_api = end_date.replace("-", "")

# Estrazione del poligono (AOI) in formato GeoJSON
polygon_conf = data_conf.get("polygon")
if not polygon_conf:
    raise ValueError("Definire il poligono nella sezione 'data: polygon'.")
try:
    from sentinelsat import geojson_to_wkt
    footprint = geojson_to_wkt(polygon_conf)
except ImportError:
    footprint = None  # Nel caso in cui non venga importata sentinelsat, il footprint resta None

# Determina il provider di download e i parametri satellitari relativi
provider = download_conf.get("provider", "copernicus").lower()

if provider == "copernicus":
    sat_params = satellite_conf.get("copernicus", {})
else:
    # Per USGS (es. dati Landsat)
    sat_params = satellite_conf.get("usgs", {})

platform = sat_params.get("platform")
product_level = sat_params.get("product_level")

# Imposta la cartella di destinazione:
# Se download.dest_folder non è definito, si usa "data/<product_level>"
dest_folder = download_conf.get("dest_folder")
if not dest_folder:
    dest_folder = os.path.join("data", product_level)
if not os.path.exists(dest_folder):
    os.makedirs(dest_folder)

# ----- Branching in base al provider ----- #
if provider == "copernicus":
    print("Connessione al Copernicus Open Access Hub...")
    copernicus_conf = download_conf.get("copernicus", {})
    api_url = copernicus_conf.get("url", "https://scihub.copernicus.eu/dhus")
    username = copernicus_conf.get("username", "your_username")
    password = copernicus_conf.get("password", "your_password")
    cloud_cover = copernicus_conf.get("cloud_cover_threshold", 30)

    # Connetti all'API di Copernicus con sentinelsat
    from sentinelsat import SentinelAPI
    api = SentinelAPI(username, password, api_url)

    print(f"Esecuzione query per {platform} ({product_level}) dal {start_date} al {end_date}, con copertura nuvolosa <= {cloud_cover}%")
    products = api.query(
        footprint,
        date=(start_date_api, end_date_api),
        platformname=platform,
        cloudcoverpercentage=(0, cloud_cover)
    )
    print(f"Trovati {len(products)} prodotti.")

    if products:
        print(f"Inizio il download dei prodotti nella cartella: {dest_folder}")
        api.download_all(products, directory_path=dest_folder)
        print("Download completato!")
    else:
        print("Nessun prodotto trovato per i parametri specificati.")

elif provider == "usgs":
    print("Connessione all'API USGS per dati Landsat...")
    usgs_conf = download_conf.get("usgs", {})
    usgs_url = usgs_conf.get("url", "https://earthexplorer.usgs.gov/inventory/json/v/1.4.1/")
    api_key = usgs_conf.get("api_key", "YOUR_USGS_API_KEY")
    dataset = usgs_conf.get("dataset", "LANDSAT_8_C1")

    print(f"Parametri USGS: URL={usgs_url}, API key=********, Dataset={dataset}")
    print(f"Esecuzione query per immagini del dataset {dataset} dal {start_date} al {end_date}...")
    
    # Esempio: simulazione della query e download dei dati USGS
    dummy_products = [
        {"id": "usgs_001", "title": f"{dataset}_20220101", "filename": f"{dataset}_20220101.zip"},
        {"id": "usgs_002", "title": f"{dataset}_20220115", "filename": f"{dataset}_20220115.zip"},
        {"id": "usgs_003", "title": f"{dataset}_20220131", "filename": f"{dataset}_20220131.zip"}
    ]
    print(f"Trovati {len(dummy_products)} prodotti (simulati).")
    
    print("Inizio il download (simulato) dei prodotti USGS...")
    for prod in dummy_products:
        file_path = os.path.join(dest_folder, prod["filename"])
        with open(file_path, "w") as f:
            f.write(f"File fittizio per il prodotto {prod['title']}")
        print(f"Mock downloaded: {file_path}")
    print("Simulazione download USGS completata!")

else:
    print("Provider di download non riconosciuto. Verifica il campo 'provider' nel file di configurazione.")

print("Fine dello script.")


Connessione all'API USGS per dati Landsat...
Parametri USGS: URL=https://earthexplorer.usgs.gov/inventory/json/v/1.4.1/, API key=********, Dataset=LANDSAT_8_C1
Esecuzione query per immagini del dataset LANDSAT_8_C1 dal 2022-01-01 al 2022-01-31...
Trovati 3 prodotti (simulati).
Inizio il download (simulato) dei prodotti USGS...
Mock downloaded: data\L1\LANDSAT_8_C1_20220101.zip
Mock downloaded: data\L1\LANDSAT_8_C1_20220115.zip
Mock downloaded: data\L1\LANDSAT_8_C1_20220131.zip
Simulazione download USGS completata!
Fine dello script.


c:\Users\fgalassi\Progetti\EO Pipeline\processing-root-folder\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
